# Python爬虫 Urllib的高级用法

## 1.设置Headers

如果识别有问题，那么站点根本不会响应，所以为了完全模拟浏览器的工作，我们需要设置一些Headers 的属性。

首先，打开我们的浏览器，调试浏览器F12，我用的是Chrome，打开网络监听，示意如下，比如知乎，点登录之后，我们会发现登陆之后界面都变化 了，出现一个新的界面，实质上这个页面包含了许许多多的内容，这些内容也不是一次性就加载完成的，实质上是执行了好多次请求，一般是首先请求HTML文 件，然后加载JS，CSS 等等，经过多次请求之后，网页的骨架和肌肉全了，整个网页的效果也就出来了。

拆分这些请求，我们只看一第一个请求，你可以看到，有个Request URL，还有headers，下面便是response，图片显示得不全，小伙伴们可以亲身实验一下。那么这个头中包含了许许多多是信息，有文件编码啦，压缩方式啦，请求的agent啦等等。

其中，agent就是请求的身份，如果没有写入请求身份，那么服务器不一定会响应，所以可以在headers中设置agent,例如下面的例子，这个例子只是说明了怎样设置的headers，小伙伴们看一下设置格式就好。


In [9]:
#import urllib
#import urllib2

#url = "https://www.zhihu.com/#signin"
#user_agent = 'Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)'
#values = {'user_login' : 'wuyunxing@hotmail.com','user_password' : 'wyx83695519' } 
#headers = { 'User-Agent' : user_agent }  
#data = urllib.urlencode(values)  
#request = urllib2.Request(url, data, headers)  
#response = urllib2.urlopen(request)  
#page = response.read()

这样，我们设置了一个headers，在构建request时传入，在请求时，就加入了headers传送，服务器若识别了是浏览器发来的请求，就会得到响应。

另外，我们还有对付”反盗链”的方式，对付防盗链，服务器会识别headers中的referer是不是它自己，如果不是，有的服务器不会响应，所以我们还可以在headers中加入referer

例如我们可以构建下面的headers

In [10]:
headers = { 'User-Agent' : 'Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)'  ,
                        'Referer':'http://www.zhihu.com/articles' }

同上面的方法，在传送请求时把headers传入Request参数里，这样就能应付防盗链了。

另外headers的一些属性，下面的需要特别注意一下：

> User-Agent : 有些服务器或 Proxy 会通过该值来判断是否是浏览器发出的请求

> Content-Type : 在使用 REST 接口时，服务器会检查该值，用来确定 HTTP Body 中的内容该怎样解析。

> application/xml ： 在 XML RPC，如 RESTful/SOAP 调用时使用

> application/json ： 在 JSON RPC 调用时使用

> application/x-www-form-urlencoded ： 浏览器提交 Web 表单时使用在使用服务器提供的 RESTful 或 SOAP 服务时， Content-Type 设置错误会导致服务器拒绝服务

其他的有必要的可以审查浏览器的headers内容，在构建时写入同样的数据即可。

## 2. Proxy(代理)的设置

urllib2 默认会使用环境变量 http_proxy 来设置 HTTP Proxy。假如一个网站它会检测某一段时间某个IP 的访问次数，如果访问次数过多，它会禁止你的访问。所以你可以设置一些代理服务器来帮助你做工作，每隔一段时间换一个代理，网站君都不知道是谁在捣鬼了，这酸爽！

In [11]:
#import urllib2
#enable_proxy = True
#proxy_handler = urllib2.ProxyHandler({"http" : 'http://some-proxy.com:8080'})
#null_proxy_handler = urllib2.ProxyHandler({})
#if enable_proxy:
#    opener = urllib2.build_opener(proxy_handler)
#else:
#    opener = urllib2.build_opener(null_proxy_handler)
#urllib2.install_opener(opener)

## 3.Timeout 设置

上一节已经说过urlopen方法了，第三个参数就是timeout的设置，可以设置等待多久超时，为了解决一些网站实在响应过慢而造成的影响。

例如下面的代码,如果第二个参数data为空那么要特别指定是timeout是多少，写明形参，如果data已经传入，则不必声明。

In [13]:
import urllib2
response = urllib2.urlopen('http://www.baidu.com', timeout=10)

import urllib2
response = urllib2.urlopen('http://www.baidu.com',data, 10)

## 4.使用 HTTP 的 PUT 和 DELETE 方法

http协议有六种请求方法，get,head,put,delete,post,options，我们有时候需要用到PUT方式或者DELETE方式请求。

> PUT：这个方法比较少见。HTML表单也不支持这个。本质上来讲， PUT和POST极为相似，都是向服务器发送数据，但它们之间有一个重要区别，PUT通常指定了资源的存放位置，而POST则没有，POST的数据存放位置由服务器自己决定。

> DELETE：删除某一个资源。基本上这个也很少见，不过还是有一些地方比如amazon的S3云服务里面就用的这个方法来删除资源。

如果要使用 HTTP PUT 和 DELETE ，只能使用比较低层的 httplib 库。虽然如此，我们还是能通过下面的方式，使 urllib2 能够发出 PUT 或DELETE 的请求，不过用的次数的确是少，在这里提一下。

In [14]:
#import urllib2
#request = urllib2.Request(uri, data=data)
#request.get_method = lambda: 'PUT' # or 'DELETE'
#response = urllib2.urlopen(request)

## 5.使用DebugLog

可以通过下面的方法把 Debug Log 打开，这样收发包的内容就会在屏幕上打印出来，方便调试，这个也不太常用，仅提一下

In [15]:
import urllib2
httpHandler = urllib2.HTTPHandler(debuglevel=1)
httpsHandler = urllib2.HTTPSHandler(debuglevel=1)
opener = urllib2.build_opener(httpHandler, httpsHandler)
urllib2.install_opener(opener)
response = urllib2.urlopen('http://www.baidu.com')

send: 'GET / HTTP/1.1\r\nAccept-Encoding: identity\r\nHost: www.baidu.com\r\nConnection: close\r\nUser-Agent: Python-urllib/2.7\r\n\r\n'
reply: 'HTTP/1.1 200 OK\r\n'
header: Date: Thu, 19 May 2016 16:43:44 GMT
header: Content-Type: text/html; charset=utf-8
header: Transfer-Encoding: chunked
header: Connection: Close
header: Vary: Accept-Encoding
header: Set-Cookie: BAIDUID=25F5064D8020D819E409EDA2E500F1DD:FG=1; expires=Thu, 31-Dec-37 23:55:55 GMT; max-age=2147483647; path=/; domain=.baidu.com
header: Set-Cookie: BIDUPSID=25F5064D8020D819E409EDA2E500F1DD; expires=Thu, 31-Dec-37 23:55:55 GMT; max-age=2147483647; path=/; domain=.baidu.com
header: Set-Cookie: PSTM=1463676224; expires=Thu, 31-Dec-37 23:55:55 GMT; max-age=2147483647; path=/; domain=.baidu.com
header: Set-Cookie: BDSVRTM=0; path=/
header: Set-Cookie: BD_HOME=0; path=/
header: Set-Cookie: H_PS_PSSID=19289_1434_18280_19558_19860_15068_11924; path=/; domain=.baidu.com
header: P3P: CP=" OTI DSP COR IVA OUR IND COM "
header: Cache

以上便是一部分高级特性，前三个是重要内容，在后面，还有cookies的设置还有异常的处理，小伙伴们加油！

# Python爬虫 URLError异常处理

## 1.URLError

首先解释下URLError可能产生的原因:

* 网络无连接，即本机无法上网
* 连接不到指定的服务器
* 服务器不存在

在代码中，我们需要用try-except语句来包围并捕获相应的异常。下面是一个例子：

In [20]:
import urllib2

requset = urllib2.Request('http://www.yufytf.com')

try:
    urllib2.urlopen(requset)
except urllib2.URLError, e:
    print e.reason

[Errno 8] nodename nor servname provided, or not known


## 2.HTTPError

HTTPError是URLError的子类，在你利用urlopen方法发出一个请求时，服务器上都会对应一个应答对象response，其中它包含一个数字”状态码”。举个例子，假如response是一个”重定向”，需定位到别的地址获取文档，urllib2将对此进行处理。

其他不能处理的，urlopen会产生一个HTTPError，对应相应的状态吗，HTTP状态码表示HTTP协议所返回的响应的状态。下面将状态码归结如下：

> 100：继续 客户端应当继续发送请求。客户端应当继续发送请求的剩余部分，或者如果请求已经完成，忽略这个响应。

> 101： 转换协议 在发送完这个响应最后的空行后，服务器将会切换到在Upgrade 消息头中定义的那些协议。只有在切换新的协议更有好处的时候才应该采取类似措施。

> 102：继续处理 由WebDAV（RFC 2518）扩展的状态码，代表处理将被继续执行。

> 200：请求成功 处理方式：获得响应的内容，进行处理

> 201：请求完成，结果是创建了新资源。新创建资源的URI可在响应的实体中得到 处理方式：爬虫中不会遇到

> 202：请求被接受，但处理尚未完成 处理方式：阻塞等待

> 204：服务器端已经实现了请求，但是没有返回新的信 息。如果客户是用户代理，则无须为此更新自身的文档视图。 处理方式：丢弃

> 300：该状态码不被HTTP/1.0的应用程序直接使用， 只是作为3XX类型回应的默认解释。存在多个可用的被请求资源。 处理方式：若程序中能够处理，则进行进一步处理，如果程序中不能处理，则丢弃

> 301：请求到的资源都会分配一个永久的URL，这样就可以在将来通过该URL来访问此资源 处理方式：重定向到分配的URL

> 302：请求到的资源在一个不同的URL处临时保存 处理方式：重定向到临时的URL

> 304：请求的资源未更新 处理方式：丢弃

> 400：非法请求 处理方式：丢弃

> 401：未授权 处理方式：丢弃

> 403：禁止 处理方式：丢弃

> 404：没有找到 处理方式：丢弃

> 500：服务器内部错误 服务器遇到了一个未曾预料的状况，导致了它无法完成对请求的处理。一般来说，这个问题都会在服务器端的源代码出现错误时出现。

> 501：服务器无法识别 服务器不支持当前请求所需要的某个功能。当服务器无法识别请求的方法，并且无法支持其对任何资源的请求。

> 502：错误网关 作为网关或者代理工作的服务器尝试执行请求时，从上游服务器接收到无效的响应。

> 503：服务出错 由于临时的服务器维护或者过载，服务器当前无法处理请求。这个状况是临时的，并且将在一段时间以后恢复。

HTTPError实例产生后会有一个code属性，这就是是服务器发送的相关错误号。
因为urllib2可以为你处理重定向，也就是3开头的代号可以被处理，并且100-299范围的号码指示成功，所以你只能看到400-599的错误号码。

下面我们写一个例子来感受一下，捕获的异常是HTTPError，它会带有一个code属性，就是错误代号，另外我们又打印了reason属性，这是它的父类URLError的属性。

In [21]:
import urllib2
 
req = urllib2.Request('http://blog.csdn.net/cqcre')
try:
    urllib2.urlopen(req)
except urllib2.HTTPError, e:
    print e.code
    print e.reason

send: 'GET /cqcre HTTP/1.1\r\nAccept-Encoding: identity\r\nHost: blog.csdn.net\r\nConnection: close\r\nUser-Agent: Python-urllib/2.7\r\n\r\n'
reply: 'HTTP/1.1 403 Forbidden\r\n'
header: Server: openresty
header: Date: Thu, 19 May 2016 16:53:38 GMT
header: Content-Type: text/html; charset=utf-8
header: Content-Length: 162
header: Connection: close
403
Forbidden


错误代号是403，错误原因是Forbidden，说明服务器禁止访问。

我们知道，HTTPError的父类是URLError，根据编程经验，父类的异常应当写到子类异常的后面，如果子类捕获不到，那么可以捕获父类的异常，所以上述的代码可以这么改写

In [22]:
import urllib2
 
req = urllib2.Request('http://blog.csdn.net/cqcre')
try:
    urllib2.urlopen(req)
except urllib2.HTTPError, e:
    print e.code
except urllib2.URLError, e:
    print e.reason
else:
    print "OK"

send: 'GET /cqcre HTTP/1.1\r\nAccept-Encoding: identity\r\nHost: blog.csdn.net\r\nConnection: close\r\nUser-Agent: Python-urllib/2.7\r\n\r\n'
reply: 'HTTP/1.1 403 Forbidden\r\n'
header: Server: openresty
header: Date: Thu, 19 May 2016 16:54:30 GMT
header: Content-Type: text/html; charset=utf-8
header: Content-Length: 162
header: Connection: close
403


如果捕获到了HTTPError，则输出code，不会再处理URLError异常。如果发生的不是HTTPError，则会去捕获URLError异常，输出错误原因。

另外还可以加入 hasattr属性提前对属性进行判断，代码改写如下

In [23]:
import urllib2
 
req = urllib2.Request('http://blog.csdn.net/cqcre')

try:
    urllib2.urlopen(req)
except urllib2.URLError, e:
    if hasattr(e,"code"):
        print e.code
    if hasattr(e,"reason"):
        print e.reason
else:
    print "OK"

send: 'GET /cqcre HTTP/1.1\r\nAccept-Encoding: identity\r\nHost: blog.csdn.net\r\nConnection: close\r\nUser-Agent: Python-urllib/2.7\r\n\r\n'
reply: 'HTTP/1.1 403 Forbidden\r\n'
header: Server: openresty
header: Date: Thu, 19 May 2016 16:55:54 GMT
header: Content-Type: text/html; charset=utf-8
header: Content-Length: 162
header: Connection: close
403
Forbidden


首先对异常的属性进行判断，以免出现属性输出报错的现象。

以上，就是对URLError和HTTPError的相关介绍，以及相应的错误处理办法。